# Setup

In order to call the API you'll need to get two things from the [Deploy dashboard](https://dash.deno.com/):
  * Your organization ID
  * A personal or organizational access token

In [1]:
import { assert } from "https://deno.land/std@0.202.0/testing/asserts.ts";

// Replace these with your own!
const organizationId = "a75a9caa-b8ac-47b3-a423-3f2077c58731";
const token = "ddo_u7mo08lBNHm8GMGLhtrEVfcgBsCuSp36dumX";

# Create a project

In order to create deployments, you first need to create a project to host them.

The project name is used to derive deployments URLs from. You may specify one;
if you don't, a random project name will be assigned automatically.

In [1]:
const res = await fetch(
  `https://api.deno.com/v1/organizations/${organizationId}/projects`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        name: null
      } satisfies CreateProjectRequest
    ),
  },
);
assert(res.ok);

const project = await res.json();
console.log(project);

{
  id: "d798575b-b81b-40b8-b87c-184b382e2b9e",
  name: "sad-rhino-75",
  createdAt: "2023-09-22T05:38:34.550090Z",
  updatedAt: "2023-09-22T05:38:34.550090Z"
}


# Create a deployment

A typical flow for creating a deployment is described in this section.

### 1. Initiate a build

To initiate a build, you have call the API and provide (at a minimum):
  * One or more source files
  * The (relative) URL to the entry point source file
  * A set of environment variables

In [1]:
const res = await fetch(
  `https://api.deno.com/v1/projects/${project.id}/deployments`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        "entryPointUrl": "main.ts",
        "assets": {
          "main.ts": {
            "kind": "file",
            "content":
              `import { assert } from "https://deno.land/std@0.202.0/testing/asserts.ts";
               Deno.serve((req: Request) => new Response('Hello World'));`
          }
        },
        "envVars": {
          "FOO": "foo",
        },
      } satisfies CreateDeploymentRequest,
    ),
  },
);
assert(res.ok);

const deployment: Deployment = await res.json();
console.log(deployment);

{
  id: "rn3mjpwsw24d",
  projectId: "d798575b-b81b-40b8-b87c-184b382e2b9e",
  status: "pending",
  createdAt: "2023-09-22T05:54:38.007146Z",
  updatedAt: "2023-09-22T05:54:38.007146Z"
}


### 2. Wait for the build to be completed

Build of a deployment is an async process. In order to wait for its completion (or failure), you may either:
  * Use the `GET /v1/deployments/:deploymentId/build_logs` endpoint. This endpoint will stream build log messages until the build is complete.
  * Periodically poll the `GET /v1/deployments/:deploymentId` endpoint until the response `status` field is no longer `pending`.


In [1]:
import { DelimiterStream } from "https://deno.land/std@0.202.0/streams/delimiter_stream.ts";

const res = await fetch(
  `https://api.deno.com/v1/deployments/${deployment.id}/build_logs`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
      // The endpoint supports two different formats: `application/json` and
      // `application/x-ndjson`; the latter is easier to parse incrementally.
      accept: "application/x-ndjson",
    },
  },
);
assert(res.ok);

const lines = res.body!
  .pipeThrough(new DelimiterStream(new TextEncoder().encode("\n")))
  .pipeThrough(new TextDecoderStream());

for await (const line of lines) {
  const logEntry: BuildLogEntry = JSON.parse(line);
  console.log(`[${logEntry.level}] ${logEntry.message}`);
}

[info] Downloaded file:///src/main.ts (1/2)
[info] Downloaded https://deno.land/std@0.202.0/testing/asserts.ts (2/3)
[info] Downloaded https://deno.land/std@0.202.0/assert/mod.ts (3/30)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_almost_equals.ts (4/30)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_array_includes.ts (5/31)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_equals.ts (6/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_exists.ts (7/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_false.ts (8/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_greater_or_equal.ts (9/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_greater.ts (10/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_instance_of.ts (11/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_is_error.ts (12/34)
[info] Downloaded https://deno.land/std@0.202.0/assert/assert_less_o

### 3. Obtain deployment metadata

Once the build is completed, the deployment's status will change to `success`, and a deno.dev domain will be automatically assigned to it.

In [1]:
const res = await fetch(
  `https://api.deno.com/v1/deployments/${deployment.id}`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assert(res.ok);

const deployment2: Deployment = await res.json();
console.log(deployment2);

{
  id: "rn3mjpwsw24d",
  projectId: "d798575b-b81b-40b8-b87c-184b382e2b9e",
  status: "success",
  domains: [ "sad-rhino-75-rn3mjpwsw24d.deno.dev" ],
  createdAt: "2023-09-22T05:54:38.007146Z",
  updatedAt: "2023-09-22T05:54:43.720061Z"
}


# Link a custom domain to your account

The next section outlines how to perform domain-related operations:
  * Link a domain name to your account
  * Verify that you own the domain name
  * Provision TLS certificates

### 1. Link a domain


In [1]:
const res = await fetch(
  `https://api.deno.com/v1/organizations/${organizationId}/domains`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        // Replace with a domain you own!
        "domain": "deploy-sample1.maguro.me",
      } satisfies CreateDomainRequest,
    ),
  },
);
assert(res.ok);

const domain: Domain = await res.json();
console.log(domain);

{
  id: "8302bc00-0fe3-4416-a190-ead9b48d9ce9",
  domain: "deploy-sample1.maguro.me",
  token: "6a30d42ce0da4d5013b5e39c",
  isValidated: false,
  certificates: [],
  provisioningStatus: { code: "pending" },
  projectId: "df5c078b-4639-40c3-b3b7-ca05e81edb89",
  createdAt: "2023-09-04T01:54:28.199638Z",
  updatedAt: "2023-09-05T00:26:15.284119Z",
  dnsRecords: [
    { type: "A", name: "deploy-sample1", content: "34.120.54.55" },
    {
      type: "AAAA",
      name: "deploy-sample1",
      content: "2600:1901:0:6d85::"
    },
    {
      type: "CNAME",
      name: "_acme-challenge.deploy-sample1",
      content: "6a30d42ce0da4d5013b5e39c._acme.deno.dev."
    }
  ]
}


### 2. Verify the domain ownership

Once you linked your domain, Deno Deploy needs to verify that you are the owner of the domain. You can do so by following these steps:

- Retrieve the `dnsRecords` field from the resonse body of the API call above. Notice that it includes three values: type `A`, `AAAA`, and `CNAME`.
- Add these DNS records to the DNS provider you're using for this domain. Here's my setting on Cloudflare dashboard:
![How I configured DNS records on Cloudflare dashboard](https://github.com/magurotuna/zig-deque/assets/23649474/5bfeaafc-58c0-46fa-857c-ec33fdf3f3a8)
- Call the API to get Deploy to verify the expected records are set:

In [ ]:
const res = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}/verify`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assert(res.ok);

### 3. Provision TLS certificates

Now it's verified that you are the owner of the domain. Final step is to provision TLS certificates for the domain. You can either upload the certificates you prepared somehow, or just leave the certificates preparation to Deploy. Here we will demonstrate the latter, auto-provisioning capability.

All you have to do is to call `POST /v1/domains/:domainId/certificates/provision` which triggers the auto-provisioning step for the specified domain.

In [ ]:
// This call may take a while, up to a minute or so.
const res = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}/certificates/provision`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assert(res.ok);

Just to make sure, let's call `GET /v1/domains/:domainId` to see the current state of the domain.

In [1]:
const res = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assert(res.ok);

const domain: Domain = await res.json();
console.log(domain);

{
  id: "8302bc00-0fe3-4416-a190-ead9b48d9ce9",
  domain: "deploy-sample1.maguro.me",
  token: "6a30d42ce0da4d5013b5e39c",
  isValidated: true,
  certificates: [
    {
      cipher: "rsa",
      expiresAt: "2023-12-03T23:26:38.445898Z",
      createdAt: "2023-09-05T00:26:40.449996Z",
      updatedAt: "2023-09-05T00:26:40.449996Z"
    },
    {
      cipher: "ec",
      expiresAt: "2023-12-03T23:26:39.992532Z",
      createdAt: "2023-09-05T00:26:41.995657Z",
      updatedAt: "2023-09-05T00:26:41.995657Z"
    }
  ],
  provisioningStatus: { code: "success" },
  projectId: "df5c078b-4639-40c3-b3b7-ca05e81edb89",
  createdAt: "2023-09-04T01:54:28.199638Z",
  updatedAt: "2023-09-05T00:26:15.284119Z",
  dnsRecords: [
    { type: "A", name: "deploy-sample1", content: "34.120.54.55" },
    {
      type: "AAAA",
      name: "deploy-sample1",
      content: "2600:1901:0:6d85::"
    },
    {
      type: "CNAME",
      name: "_acme-challenge.deploy-sample1",
      content: "6a30d42ce0da4d5013b5e39c.

# Associate the custom domain with the deployment

Although the domain setup has completed, it is of no use until it gets connected to an actual deployment. We accomplish it by calling `PATCH /v1/domains/:domainId`.

In [ ]:
const res = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}`,
  {
    method: "PATCH",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        "deploymentId": deployment.id,
      } satisfies UpdateDomainAssociationRequest,
    ),
  },
);
assert(res.ok);

Now you can confirm that your deployment is reachable using your own domain. Here's the result of `curl` in my case:

```sh
❯ curl --include 'https://deploy-sample1.maguro.me'
HTTP/2 200
content-type: text/plain;charset=UTF-8
vary: Accept-Encoding
date: Thu, 21 Sep 2023 10:51:34 GMT
content-length: 11
via: http/2 edgeproxy
server: deno/gcp-asia-northeast1

Hello World%
```

# Appendix

Here is a list of types used in the sample code snippets above.

In [ ]:
type CreateProjectRequest = {
  name: string | null;
};

type Project = {
  id: string;
  name: string;
  organizationId: string;
  createdAt: string;
  updatedAt: string;
};

type CreateDeploymentRequest = {
  entryPointUrl: string;
  importMapUrl?: string | null;
  lockFileUrl?: string | null;
  compilerOptions?: CompilerOptions | null;
  assets: Record<string, Asset>;
  envVars: Record<string, string>;
};

type CompilerOptions = {
  jsx?: string | null;
  jsxFactory?: string | null;
  jsxFragmentFactory?: string | null;
  jsxImportSource?: string | null;
};

type Asset = FileAsset | SymlinkAsset;

type FileAsset = FileContent | FileHash;

type FileContent = {
  kind: "file";
  content: string;
  encoding?: "utf-8" | "base64";
};

type FileHash = {
  kind: "file";
  gitSha1: string;
};

type SymlinkAsset = {
  kind: "symlink";
  target: string;
};

type Deployment = {
  id: string;
  projectId: string;
  status: "failed" | "pending" | "success";
  domains?: string[];
  createdAt: string;
  updatedAt: string;
};

type BuildLogEntry = {
  level: string;
  message: string;
};

type CreateDomainRequest = {
  domain: string;
};

type Domain = {
  id: string;
  organizationId: string;
  domain: string;
  token: string;
  isValidated: boolean;
  certificates: Certificate[];
  provisioningStatus: ProvisioningStatus;
  projectId: string | null;
  createdAt: string;
  updatedAt: string;
  dnsRecords: DnsRecord[];
};

type ProvisioningStatus = 
  | { code:  "pending" } 
  | { code: "success" } 
  | { code: "failed", message: string } 
  | { code: "manual" };

type Certificate = {
  cipher: "ec" | "rsa";
  expiresAt: string;
  createdAt: string;
  updatedAt: string;
};

type DnsRecord = {
  type: string;
  name: string;
  content: string;
};

type UpdateDomainAssociationRequest = {
  deploymentId: string | null;
};